## NDAGIRE CATHERINE - M24B38/004

### EXERCISE 2:
1. At what point in the code is the user’s input appended to the conversation history?
2. What method is used to send the conversation prompt to GPT-5?
3. How does the code handle empty user inputs?
4. Modify the code so that the assistant’s replies are limited to at least 15 words and at most 50 words.
5. Update the script to allow the user to save the entire conversation history into a .txt file once they type save.
6. Add error handling so that if the API call fails, the program retries three times before quitting.
7. Add functionality for the assistant to summarise the conversation so far if the user types summary.

#### Number 1

The user's input is appended to the conversation history at the line history.append(f"User: {user_msg}"). This happens after the code checks if the user's message is empty and before the prompt is sent to the API.

#### Number 2

The code uses the client.responses.create() method to send the conversation prompt to the GPT-5 model.

#### Number 3

The code handles empty user inputs with a if not user_msg: continue check. If the user's input is an empty string, this line tells the program to skip the rest of the loop and immediately move on to the next iteration, prompting the user for a new message.

#### Updated Code

In [ ]:
# Requires: openai >= 1.60.0
# Run: pip install --upgrade openai

from openai import OpenAI
import time

# Create an OpenAI client (replace with your real key or env var)
client = OpenAI(api_key="Your OPENAI Key Here")

# Define system prompt with response length constraints
system_prompt = "You are an intelligent assistant. You must limit your responses to at least 15 words and at most 50 words."

# Keep every turn as a simple string in `history`
history = [f"System: {system_prompt}"]

# Interactive loop
print("Type your messages. Type 'bye' to exit.")
print("Type 'save' to save the conversation. Type 'summary' to get a summary.\n")

#Number 5 and Number 7
# GPT-5 interactive chat with save and summary features
while True:
    user_msg = input("User: ").strip()
    if not user_msg:
        continue
    if user_msg.lower() == "bye":
        print("GPT-5: Goodbye 👋")
        break
    
    if user_msg.lower() == "save":
        with open("conversation_history.txt", "w") as f:
            f.write("\n".join(history))
        print("Conversation saved to conversation_history.txt\n")
        continue

    if user_msg.lower() == "summary":
        # Create a new prompt for the summary request
        summary_prompt = "\n".join(history) + "\n\nUser: Please summarize the conversation so far."
        try:
            response = client.responses.create(
                model="gpt-5",
                input=summary_prompt,
                reasoning={"effort": "medium"},
                text={"verbosity": "medium"}
            )
            summary_text = response.output_text.strip()
            print(f"GPT-5 Summary: {summary_text}\n")
        except Exception as e:
            print(f"Error getting summary: {e}")
        continue

    # Save the user input
    history.append(f"User: {user_msg}")

    # Join the transcript into one big text block
    prompt = "\n".join(history)

    #Error handling so that if the API call fails, the program retries three times before giving up
    # Retry logic
    retries = 0
    max_retries = 3
    while retries < max_retries:
        try:
            # Send the prompt to GPT-5
            response = client.responses.create(
                model="gpt-5",
                input=prompt,
                reasoning={"effort": "medium"},
                text={"verbosity": "medium"}
            )
            
            # Get the assistant’s text
            reply = response.output_text.strip()
            
            # Save assistant reply for context in the next turn
            history.append(f"Assistant: {reply}")
            print(f"GPT-5: {reply}\n")
            break  # Break out of the retry loop on success

        except Exception as e:
            print(f"Error: {e}. Retrying... ({retries + 1}/{max_retries})")
            retries += 1
            time.sleep(2)  # Wait a bit before retrying
    
    if retries == max_retries:
        print("Max retries exceeded. Exiting.")
        break